<a href="https://colab.research.google.com/github/dakilaledesma/FSUS_Tools/blob/main/notebooks/GBIF_Balanced_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget https://api.gbif.org/v1/occurrence/download/request/0426301-210914110416597.zip

--2022-08-18 16:47:14--  https://api.gbif.org/v1/occurrence/download/request/0426301-210914110416597.zip
Resolving api.gbif.org (api.gbif.org)... 130.225.43.2
Connecting to api.gbif.org (api.gbif.org)|130.225.43.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://occurrence-download.gbif.org/occurrence/download/request/0426301-210914110416597.zip [following]
--2022-08-18 16:47:14--  https://occurrence-download.gbif.org/occurrence/download/request/0426301-210914110416597.zip
Resolving occurrence-download.gbif.org (occurrence-download.gbif.org)... 130.225.43.36
Connecting to occurrence-download.gbif.org (occurrence-download.gbif.org)|130.225.43.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18249057 (17M) [application/octet-stream]
Saving to: ‘0426301-210914110416597.zip’

0426301-21091411041 100%[===================>]  17.40M  13.2MB/s    in 1.3s    

2022-08-18 16:47:16 (13.2 MB/s) - ‘0426301-210914110416597.zip’ saved 

In [2]:
%%time
! unzip -q 0426301-210914110416597.zip -d /content/

CPU times: user 13.5 ms, sys: 6.83 ms, total: 20.3 ms
Wall time: 1.11 s


In [3]:
import pandas as pd

occurrence_df = pd.read_csv("occurrence.txt", delimiter="\t", encoding="latin-1", on_bad_lines="skip")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,9,26,35,39,42,46,47,54,56,57,58,59,60,61,62,64,65,66,68,69,71,74,75,76,77,79,82,84,85,87,89,90,91,92,96,98,99,101,103,109,110,111,115,116,117,119,120,121,122,123,126,127,128,129,130,132,135,136,142,143,144,145,147,148,149,150,151,152,153,154,155,156,157,158,159,160,170,171,172,173,174,175,176,177,178,179,180,182,183,189,190,195,207,210,211,212,214,215,242,246,254,255) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
name_list = occurrence_df.columns
occurrence_df = pd.read_csv("occurrence.txt", delimiter="\t", encoding="latin-1", names=name_list)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,64,65,66,68,69,71,72,73,74,75,76,77,78,79,80,81,82,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,189,190,191,192,193,194,195,201,204,205,207,208,210,211,212,214,215,219,220,221,222,223,224,227,228,229,230,231,232,233,234,235,236,237,238,242,246,247,254,255) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns

In [5]:
occurrence_df = pd.DataFrame(occurrence_df.values[1:], columns=name_list)

In [7]:
import numpy as np

sci_names = set(occurrence_df["acceptedScientificName"])
sci_counts = {}
for s in sci_names:
  sci_counts[s] = len(occurrence_df[occurrence_df["acceptedScientificName"] == s])

try:
  del sci_counts[np.nan]
except KeyError:
  pass

In [8]:
multimedia_df = pd.read_csv("multimedia.txt", delimiter="\t", encoding="latin-1")

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [10]:
id_to_sp = {}
for idx, row in occurrence_df.iterrows():
  id_to_sp[row["gbifID"]] = row["acceptedScientificName"]

In [11]:
import os
from tqdm.notebook import tqdm
from urllib import request
import time
from joblib import Parallel, delayed
from collections import defaultdict
import io
from PIL import Image
import time

rand_df = multimedia_df.sample(frac=1).reset_index(drop=True)
sample_nums = defaultdict(int)

def get_ims(idx, row):
  id = row["gbifID"]
  try:
    sp = id_to_sp[id]
  except:
    return
  im_url = row["identifier"]
  
  if sample_nums[sp] >= 20:
    return

  bn = os.path.basename(str(id))
  path = f"images/{sp}"
  try:
    req = request.urlopen(request.Request(im_url)).read()
  except:
    return

  try:
    im = Image.open(io.BytesIO(req))
    im.thumbnail([1250, 1250])
    
    if not os.path.isdir(path):
      os.makedirs(path)

    im.save(f"{path}/{id}.jpg")
    sample_nums[sp] += 1
  except:
    pass

  time.sleep(1)
  return

Parallel(n_jobs=8, require="sharedmem")(delayed(get_ims)(idx, row) for idx, row in tqdm(list(rand_df.iterrows()), total=len(rand_df)))

  0%|          | 0/76407 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 26. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 14 bytes but only got 8. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [13]:
from glob import glob

print(len(list(glob("images/**/*.*", recursive=True))))

1242


In [12]:
! zip -r -q /content/drive/MyDrive/UNC/S2022P/GBIF_balanced20_quercus.zip /content/images